<a href="https://colab.research.google.com/github/siddish-reddy/Deep-Learning-with-PyTorch/blob/master/tv_script_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
!nvidia-smi

Thu Dec 12 03:20:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!mkdir data
!wget https://github.com/udacity/deep-learning-v2-pytorch/raw/master/project-tv-script-generation/data/Seinfeld_Scripts.txt -P data
!wget https://github.com/udacity/deep-learning-v2-pytorch/raw/master/project-tv-script-generation/helper.py
!wget https://github.com/udacity/deep-learning-v2-pytorch/raw/master/project-tv-script-generation/problem_unittests.py

--2019-12-12 03:20:49--  https://github.com/udacity/deep-learning-v2-pytorch/raw/master/project-tv-script-generation/data/Seinfeld_Scripts.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/project-tv-script-generation/data/Seinfeld_Scripts.txt [following]
--2019-12-12 03:20:55--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/project-tv-script-generation/data/Seinfeld_Scripts.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3471475 (3.3M) [text/plain]
Saving to: ‘data/Seinfeld_Scripts.txt’

Seinfeld_Scripts.tx 100%[==============

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [4]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [0]:
len(text.split())

605614

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [5]:
import problem_unittests as tests
from collections import Counter
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    words = set(text)
    int_to_vocab = {i: word for i, word in enumerate(words)}
    vocab_to_int = {word: i for i, word in int_to_vocab.items()}

    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    return {
        '.': "||period||", 
        ',': "||comma||",
        '"': "||quotation||",
        ';': "||semicolon||",
        '!': "||exclamnation||",
        '?': "||question_mark||",
        '(': "||left_parentheses||",
        ')': "||right_parentheses||",
        '-': "||dash||",
        '\n': "||new_line||"
    }
        
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [0]:
len(set(int_text))

21387

In [0]:
len(vocab_to_int)

21388

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [0]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    n = len(words)
    inps = []
    tgts = []
    for i in range(sequence_length,n):
      inps.append(words[i-sequence_length:i])
      tgts.append(words[i])

    dataset = TensorDataset(torch.Tensor(inps), torch.Tensor(tgts))
    # return a dataloader
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [0]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[43., 44., 45., 46., 47.],
        [17., 18., 19., 20., 21.],
        [ 1.,  2.,  3.,  4.,  5.],
        [18., 19., 20., 21., 22.],
        [13., 14., 15., 16., 17.],
        [26., 27., 28., 29., 30.],
        [28., 29., 30., 31., 32.],
        [12., 13., 14., 15., 16.],
        [14., 15., 16., 17., 18.],
        [40., 41., 42., 43., 44.]])

torch.Size([10])
tensor([48., 22.,  6., 23., 18., 31., 33., 17., 19., 45.])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [0]:
# train_on_gpu = False

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(dropout)

    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)

        embeds = self.embedding(nn_input.long())
        output, hidden = self.lstm(embeds, hidden)
        output = output.contiguous().view(-1, self.hidden_dim)
        output = self.dropout(output)
        output = self.fc(output)

        # return one batch of output word scores and the hidden state
        # reshape into (batch_size, seq_length, output_size)
        output = output.view(batch_size, -1, self.output_size)
        # get last batch
        output = output[:, -1]
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    # move data to GPU, if available
    if train_on_gpu:
      inp = inp.cuda()
      target = target.cuda()
    hidden = tuple([each.data for each in hidden])
    rnn.zero_grad()

    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target.long())
    # perform backpropagation and optimization
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
            elif batch_i%30 == 0:
              print('.', end='')
        torch.save(rnn, '/content/drive/My Drive/training_rnn.pt')        
        print('Saving Model')
      

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [0]:
# Data params
# Sequence Length
sequence_length = 8  # of words in a sequence
# Batch Size
batch_size = 32

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 4
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 400
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 1000

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 4 epoch(s)...
.................................Epoch:    1/4     Loss: 5.5113825812339785

.................................Epoch:    1/4     Loss: 5.049199531555176

.................................Epoch:    1/4     Loss: 4.912330102682113

.................................Epoch:    1/4     Loss: 4.81165660738945

.................................Epoch:    1/4     Loss: 4.722360448122025

.................................Epoch:    1/4     Loss: 4.696114155292511

.................................Epoch:    1/4     Loss: 4.673862761974335

.................................Epoch:    1/4     Loss: 4.606593492746353

.................................Epoch:    1/4     Loss: 4.63496356844902

.................................Epoch:    1/4     Loss: 4.598780711650848

.................................Epoch:    1/4     Loss: 4.553489749670029

.................................Epoch:    1/4     Loss: 4.582512929677963

.................................Epoch:    1/4     Loss: 4.606

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
.................................Epoch:    2/4     Loss: 4.424337593765178

.................................Epoch:    2/4     Loss: 4.398946365356445

.................................Epoch:    2/4     Loss: 4.39290738606453

.................................Epoch:    2/4     Loss: 4.360130962848664

.................................Epoch:    2/4     Loss: 4.386083058834076

.................................Epoch:    2/4     Loss: 4.373149930953979

.................................Epoch:    2/4     Loss: 4.386732179880142

.................................Epoch:    2/4     Loss: 4.3618215675354

.................................Epoch:    2/4     Loss: 4.403577586174011

.................................Epoch:    2/4     Loss: 4.341208042860031

.................................Epoch:    2/4     Loss: 4.4190267486572266

.................................Epoch:    2/4     Loss: 4.370510252475738

.................................Epoch:    2/4     Loss: 4.397634798526764

.

In [0]:
show_every_n_batches = 3000
optimizer = torch.optim.Adam(trained_rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
print("next 4 epochs continuation")
trained_rnn = train_rnn(trained_rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

next 4 epochs continuation
Training for 4 epoch(s)...
...................................................................................................Epoch:    1/4     Loss: 4.345454061190288

...................................................................................................Epoch:    1/4     Loss: 4.4215889414946234

...................................................................................................Epoch:    1/4     Loss: 4.414827603578567

...................................................................................................Epoch:    1/4     Loss: 4.420858095963796

...................................................................................................Epoch:    1/4     Loss: 4.4363116242885585

...................................................................................................Epoch:    1/4     Loss: 4.426083226044973

............................................................................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/4     Loss: 4.338894934061571

...................................................................................................Epoch:    2/4     Loss: 4.339360953410466

...................................................................................................Epoch:    2/4     Loss: 4.354000401417414

...................................................................................................Epoch:    2/4     Loss: 4.3566680412292484

...................................................................................................Epoch:    2/4     Loss: 4.347976991097132

...................................................................................................Epoch:    2/4     Loss: 4.362063475847244

...................................................................................................Epoch:    2/4     Loss: 4.366578084

Loss seems that bes minimum is 4.3 and network is not able to learn further anymore.

Trail 2
Increased sequence length, as average dialogue length is 5, meaningful script should atleast be from previous two sentences.

In [0]:
# updated data params
sequence_length = 15  
batch_size = 32

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    n = len(words)
    inps = []
    tgts = []
    for i in range(sequence_length,n):
      inps.append(words[i-sequence_length:i])
      tgts.append(words[i])

    dataset = TensorDataset(torch.Tensor(inps), torch.Tensor(tgts))
    # return a dataloader
    return DataLoader(dataset, batch_size=batch_size)

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 8
# Learning Rate
learning_rate = 0.001
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 512
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 8 epoch(s)...
...................................................................................................Epoch:    1/8     Loss: 5.120708445469538
...................................................................................................Epoch:    1/8     Loss: 4.687208378275235
...................................................................................................Epoch:    1/8     Loss: 4.62343295900027
...................................................................................................Epoch:    1/8     Loss: 4.65251144293944
...................................................................................................Epoch:    1/8     Loss: 4.634658617734909
...................................................................................................Epoch:    1/8     Loss: 4.511314759055773
...................................................................................................Epoch:    1/8     Loss: 4.5507

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/8     Loss: 4.541822626906201
...................................................................................................Epoch:    2/8     Loss: 4.42412507156531
...................................................................................................Epoch:    2/8     Loss: 4.46360287129879
...................................................................................................Epoch:    2/8     Loss: 4.56002525528272
...................................................................................................Epoch:    2/8     Loss: 4.560984809398651
...................................................................................................Epoch:    2/8     Loss: 4.446417024413744
...................................................................................................Epoch:    2/8     Loss: 4.474538421789805
...

Increase in sequence length didn't change loss, either the hidden layer nodes or rnn layers are not sufficient.
First lets try by increasing rnn layers

In [0]:
# updated data params
sequence_length = 15  
batch_size = 64
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 8
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 256
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 8 epoch(s)...
...................................................................................................Epoch:    1/8     Loss: 5.100501023610433
...................................................................................................Epoch:    1/8     Loss: 4.725719400326411
...................................................................................................Epoch:    1/8     Loss: 4.581890850226085
...................................................................................................Epoch:    1/8     Loss: 4.579718362569809
................................................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/8     Loss: 4.442984043272365
...................................................................................................Epoch:    2/8     Loss: 4.342043511549631
...................................................................................................Epoch:    2/8     Loss: 4.303360380331675
...................................................................................................Epoch:    2/8     Loss: 4.3478999822934465
................................................................Saving Model
...................................................................................................Epoch:    3/8     Loss: 4.293691364412319
...................................................................................................Epoch:    3/8     Loss: 4.2340818479855855
..............................................................

In [0]:

trained_rnn = train_rnn(trained_rnn, batch_size, optimizer, criterion, 5, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
...................................................................................................Epoch:    1/5     Loss: 4.008808643658956
...................................................................................................Epoch:    1/5     Loss: 4.040606661319733
...................................................................................................Epoch:    1/5     Loss: 4.020238397598266
...................................................................................................Epoch:    1/5     Loss: 4.082639326413473
................................................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/5     Loss: 4.062611625521904
...................................................................................................Epoch:    2/5     Loss: 4.026703649282456
...................................................................................................Epoch:    2/5     Loss: 4.008665522813797
...................................................................................................Epoch:    2/5     Loss: 4.071295589844386
................................................................Saving Model
...................................................................................................Epoch:    3/5     Loss: 4.049887395450682
...................................................................................................Epoch:    3/5     Loss: 4.023295912663142
................................................................

In [0]:
# updated data params
sequence_length = 15  
batch_size = 80
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 8
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 400
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 8 epoch(s)...
...................................................................................................Epoch:    1/8     Loss: 5.067484026193619
...................................................................................................Epoch:    1/8     Loss: 4.696484481573105
...................................................................................................Epoch:    1/8     Loss: 4.508959815422694
.......................................................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/8     Loss: 4.377766033117805
...................................................................................................Epoch:    2/8     Loss: 4.290310417572657
...................................................................................................Epoch:    2/8     Loss: 4.213046968619029
.......................................................................Saving Model
...................................................................................................Epoch:    3/8     Loss: 4.191427803854182
...................................................................................................Epoch:    3/8     Loss: 4.160924677769343
...................................................................................................Epoch:    3/8     Loss: 4.103096335570018
.........................................................

In [0]:
# updated data params
sequence_length = 12
batch_size = 124
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 500
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 12 epoch(s)...
...................................................................................................Epoch:    1/12    Loss: 5.0700909598668416
...................................................................................................Epoch:    1/12    Loss: 4.5698954713344575
.......................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/12    Loss: 4.320598203793673
...................................................................................................Epoch:    2/12    Loss: 4.199161812782288
.......................................Saving Model
...................................................................................................Epoch:    3/12    Loss: 4.113447501878825
...................................................................................................Epoch:    3/12    Loss: 4.053100499073665
.......................................Saving Model
...................................................................................................Epoch:    4/12    Loss: 4.000639025574931
...................................................................................................Epoch:    4/12    Loss: 3.964353135347366
.....................................

In [0]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 800
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 1500

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-3-12-512-800.pt')

Training for 12 epoch(s)...
.................................................Epoch:    1/12    Loss: 6.046411157290141
.................................................Epoch:    1/12    Loss: 6.398836844762166
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
.................................................Epoch:    2/12    Loss: 6.4470213327196335
.................................................Epoch:    2/12    Loss: 6.510983263015747
................Saving Model
.................................................Epoch:    3/12    Loss: 5.2809932224692835
.................................................Epoch:    3/12    Loss: 4.836697589874268
................Saving Model
.................................................Epoch:    4/12    Loss: 4.539204577284474
.................................................Epoch:    4/12    Loss: 4.398630728085836
................Saving Model
.................................................Epoch:    5/12    Loss: 4.227439351259701
.................................................Epoch:    5/12    Loss: 4.151746713002523
................Saving Model
.................................................Epoch:    6/12    Loss: 4.041730645803675
.................................................E

In [17]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 500
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-2-12-512-500.pt')

Training for 12 epoch(s)...
...................................................................................................Epoch:    1/12    Loss: 4.553756914774577
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/12    Loss: 4.1132592038774325
................Saving Model
...................................................................................................Epoch:    3/12    Loss: 3.953348437694129
................Saving Model
...................................................................................................Epoch:    4/12    Loss: 3.8580819735844565
................Saving Model
...................................................................................................Epoch:    5/12    Loss: 3.7852607585669387
................Saving Model
...................................................................................................Epoch:    6/12    Loss: 3.7312131342740065
................Saving Model
...................................................................................................Epoch:    7/12    Loss: 3.68431867

In [18]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 800
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-2-12-512-800.pt')

Training for 12 epoch(s)...
...................................................................................................Epoch:    1/12    Loss: 4.4899439829190575
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/12    Loss: 4.0588627273535485
................Saving Model
...................................................................................................Epoch:    3/12    Loss: 3.8987805044746837
................Saving Model
...................................................................................................Epoch:    4/12    Loss: 3.789687559128355
................Saving Model
...................................................................................................Epoch:    5/12    Loss: 3.7084329167956187
................Saving Model
...................................................................................................Epoch:    6/12    Loss: 3.63974576545227
................Saving Model
...................................................................................................Epoch:    7/12    Loss: 3.5898597928

In [19]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 1024
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-2-12-512-1024.pt')

Training for 12 epoch(s)...
...................................................................................................Epoch:    1/12    Loss: 4.433954535166422
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
...................................................................................................Epoch:    2/12    Loss: 4.01950337938819
................Saving Model
...................................................................................................Epoch:    3/12    Loss: 3.860781977904787
................Saving Model
...................................................................................................Epoch:    4/12    Loss: 3.748009020233811
................Saving Model
...................................................................................................Epoch:    5/12    Loss: 3.6519015258954397
................Saving Model
...................................................................................................Epoch:    6/12    Loss: 3.5879200211759277
................Saving Model
...................................................................................................Epoch:    7/12    Loss: 3.52167460141

In [20]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 1024
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 2000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-3-12-512-1024.pt')

Training for 12 epoch(s)...
..................................................................Epoch:    1/12    Loss: 4.891534395337104
..................................................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
..................................................................Epoch:    2/12    Loss: 4.278440589061698
..................................................Saving Model
..................................................................Epoch:    3/12    Loss: 4.037532839586212
..................................................Saving Model
..................................................................Epoch:    4/12    Loss: 3.8997302398889677
..................................................Saving Model
..................................................................Epoch:    5/12    Loss: 3.7936992864931764
..................................................Saving Model
..................................................................Epoch:    6/12    Loss: 3.716214475552332
..................................................Saving Model
..................................................................Epoch:    7/12    Loss: 3.6416462509284475
.....................

In [21]:
# updated data params
sequence_length = 12
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 1300
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 1500

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-3-12-512-1300.pt')

Training for 12 epoch(s)...
.................................................Epoch:    1/12    Loss: 4.867983925978343
.................................................Epoch:    1/12    Loss: 4.410841669082641
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
.................................................Epoch:    2/12    Loss: 4.189883946651412
.................................................Epoch:    2/12    Loss: 4.102362241109212
................Saving Model
.................................................Epoch:    3/12    Loss: 3.9840907019472893
.................................................Epoch:    3/12    Loss: 3.9429898352622987
................Saving Model
.................................................Epoch:    4/12    Loss: 3.8512491313440185
.................................................Epoch:    4/12    Loss: 3.8249508172671
................Saving Model
.................................................Epoch:    5/12    Loss: 3.7395390391109453
.................................................Epoch:    5/12    Loss: 3.7441763332684834
................Saving Model
.................................................Epoch:    6/12    Loss: 3.662627755754417
.................................................

In [31]:
# updated data params
sequence_length = 10
batch_size = 256
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 12
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 400
hidden_dim = 800
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 1500

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-2-10-512-1024.pt')

Training for 12 epoch(s)...
.................................................Epoch:    1/12    Loss: 4.650191651185353
.................................................Epoch:    1/12    Loss: 4.252984056154887
................

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
.................................................Epoch:    2/12    Loss: 4.05962851956006
.................................................Epoch:    2/12    Loss: 3.994299249013265
................Saving Model
.................................................Epoch:    3/12    Loss: 3.8849988359837764
.................................................Epoch:    3/12    Loss: 3.8616348552703856
................Saving Model
.................................................Epoch:    4/12    Loss: 3.7749805560996457
.................................................Epoch:    4/12    Loss: 3.7700108715693155
................Saving Model
.................................................Epoch:    5/12    Loss: 3.682170631183732
.................................................Epoch:    5/12    Loss: 3.6990057711601256
................Saving Model
.................................................Epoch:    6/12    Loss: 3.6122459963925424
...............................................

In [36]:

sequence_length = 12
batch_size = 512
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 11
learning_rate = 0.001

rnn = torch.load('/content/drive/My Drive/training_rnn-2-12-512-1024.pt')
if train_on_gpu:
    rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
print("Training better model for few more epochs ")
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, 1500)
torch.save(trained_rnn, '/content/drive/My Drive/trained_rnn-2-12-512-1024.pt')
# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training better model for few more epochs 
Training for 11 epoch(s)...
.................................................Epoch:    1/11    Loss: 3.309731337865194
........

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pack

Saving Model
.................................................Epoch:    2/11    Loss: 3.2679638668261766
........Saving Model
.................................................Epoch:    3/11    Loss: 3.213494565128322
........Saving Model
.................................................Epoch:    4/11    Loss: 3.157023896716629
........Saving Model
.................................................Epoch:    5/11    Loss: 3.1107834548818807
........Saving Model
.................................................Epoch:    6/11    Loss: 3.0667231141768148
........Saving Model
.................................................Epoch:    7/11    Loss: 3.0295530949065936
........Saving Model
.................................................Epoch:    8/11    Loss: 2.9930053686575557
........Saving Model
.................................................Epoch:    9/11    Loss: 2.9601408374569584
........Saving Model
.................................................Epoch:   10/11    Loss: 2.9332815106

In [0]:
# updated data params
sequence_length = 12
batch_size = 64
train_loader = batch_data(int_text, sequence_length, batch_size)
num_epochs = 15
learning_rate = 0.001
vocab_size = len(vocab_to_int)
output_size = vocab_size
embedding_dim = 512
hidden_dim = 1024
# Number of RNN Layers
n_layers = 2
# Show stats for every n number of batches
show_every_n_batches = 3000

rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')
torch.save(trained_rnn, '/content/drive/My Drive/training_rnn-2-12-512-1024-b64.pt')

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 


---
| rnn_layers | seq_len | embeds | hidden | loss |
|------------|---------|--------|--------|------|
| 2          | 8       | 400    | 256    | 4.3  |
| 2          | 8       | 512    | 256    | 4.4  |
| 3          | 15      | 512    | 256    | 4.0  |
| 3          | 15      | 512    | 400    | 3.8  |
| 3          | 12      | 512    | 500    | 3.7  |
| 3          | 12      | 512    | 800    | 3.5  |
| 2          | 12      | 512    | 800    | 3.4  |
|**2**       | **12**  | **512**|**1024**|**2.9**|
| 2          | 10      | 512    | 1024   | 3.4  |

Those were the trails with different hyperparameters. Sequence lenghts from 10 to 15 converged fastly in initial epochs. Finally choose twice of average length of each line as I guess that next word will atleast depend one last two lines in conversation. Hidden dimensions choosen less, the loss won't decrease much but they can't be more than 1200 or because of overfitting chance, which we can observe from generated script. Number of lstm layers with 2 seems better than 3 when other parameters are kept same. Trying with few more combinations might help to find model below 2.8, but generating sentence's meaning won't change much by 0.2 change in loss.


---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = torch.load('/content/drive/My Drive/training_rnn-2-10-512-1024.pt')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [38]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer: and you don't care what i am? you know...

elaine: well i guess it's just the same.

jerry:(to george) well, you better catch it.

jerry:(to elaine) what?

kramer: well, i got to see the pig- man.

george: i can't believe it. i know i could...

george: you know, you know, the thing i know. i was in the shower with a little hot, and i was wondering if i have to go, and i'm sorry.

elaine:(pause) well, i guess i have to go back to work.

jerry:(to george) you know what? you have an idea?

george: what?

jerry: i don't know...

elaine: you don't understand. you know, i think we could do that.

jerry: well, i don't care about it.

kramer: well, i don't know how to do it.

george: well, i was thinking of donating the bill for you.

george: what?

jerry: i don't understand, i don't care.

kramer: i thought we was going to spend a couple of weeks.

elaine:(looking at the counter) i think i can get it up to him.(to jerry) you know, i think it's really good.(to elaine) so, what are you 

In [39]:
# run the cell multiple times to get different results!
gen_length = 300 # modify the length to your preference
prime_word = 'george' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

george:...

george: oh!

jerry:(to george) i told you, i don't have a square.

jerry: i don't care for the blind.

george: what?

kramer: the woman, the woman that affected you to the hospital...

elaine:(interrupting) i don't want to hear this.

george: i can't believe you don't.

elaine: i don't know.

kramer:(laughs) i don't want it.

elaine: well, what about you?

george: i think i saw him. i was just trying to keep it to the doorman and i get the hell out of there.

george:(to jerry) oh, no, no, no.

jerry:(to george) so? what do you want to go in the bathroom?

kramer: well, i was just curious.

jerry:(sarcastic) oh, you know.

elaine: what about the drake?

jerry: i don't know. she just got to see him tomorrow afternoon, and she asked me to give him a call.

george:(to jerry and george) what is that?!

jerry: what are you doing here?

kramer: well, i'm going to have to go to work.

elaine: oh yeah, yeah.(takes a bite of paper)

george:(to elaine) oh, no. i don't know, i don't th

In [40]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry:...

george:(animated) well, i guess i better get some coffee on this, huh?(elaine nods) i can't believe i was in my house. i was just wondering if you were saying that we were both cracking around the corner.

jerry:(to kramer) i don't believe this.

jerry: i don't know how to work this.(she leaves, and walks over to the booth.)

george: i thought it was a scratch!

jerry:(to george) i don't think i know what you mean. you want a protein shake?

george: i think it's the only chance i can do with that...

george:(interrupting) well, what if i just got the job?

kramer: well, i got it.

jerry: what? what?

kramer: well, i was just talking to you. i was wondering...

jerry: oh, yeah.

george:(looking over her resume) well, you know, if i had a relationship, i was just curious if we had a fight, i was wondering.

jerry: well, i was just thinking that i'm going to get some sleep.

elaine: i don't know what the big deal is. i don't know where to go.

jerry: well, you know, it's just a

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [0]:
# save script to a text file
f =  open("/content/drive/My Drive/generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.